# **Purpose: Collect News Article titles about a currency pair and get its associated price movement (Up or Down) **

****

In [6]:
!pip install beautifulsoup4 requests
# I made these changes bitch

**Given a currency pair, get relevant news articles from the previous day**

In [7]:
from bs4 import BeautifulSoup as bs
import pandas as pd
from urllib.request import Request, urlopen
from tqdm import tqdm

class fx_news_scraper():
    def __init__(self):
        self.number_of_pages = 29
        self.news_list = []
        self.target_url = 'https://www.investing.com/news/forex-news/'
        self.output_csv = 'app/data/fx-news-archive_test.csv'
        self.news_dataset = pd.DataFrame()

    def scraper(self):
        for page_no in tqdm(range(1, self.number_of_pages+1)):
            url = self.target_url + str(page_no)
            req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
            
            webpage = urlopen(req).read()
            
            soup = bs(webpage, "html.parser")
            
            news_container_div = soup.find('div', class_= 'news-analysis-v2_articles-container__3fFL8 mdMax:px-3 mb-12')

            try:
                for news_div in news_container_div.find_all('div', class_= 'news-analysis-v2_content__z0iLP w-full text-xs sm:flex-1'):
                    news_title_a = news_div.find('a', class_ = 'text-inv-blue-500 hover:text-inv-blue-500 hover:underline focus:text-inv-blue-500 focus:underline whitespace-normal text-sm font-bold leading-5 !text-[#181C21] sm:text-base sm:leading-6 lg:text-lg lg:leading-7')
                    news_body    = news_div.find('p')
                    news_date    = news_div.find('time', class_ = 'mx-1 shrink-0 text-xs leading-4').get('datetime')
                    
                    news_href    = news_title_a.get('href')
                    
                    self.news_list.append([page_no, news_date, news_title_a.text , news_body.text, news_href])

                    
            except ValueError as vEr:
                print(vEr)

            
            self.news_dataset = pd.DataFrame(self.news_list, columns = ['page', 'date','title','body', 'url'] )
            self.news_dataset.to_csv(self.output_csv)
            
            page_no += 1

        print ('scraping completed')
        print (f'output --> {self.output_csv}')

In [8]:
news_scraper = fx_news_scraper()
news_scraper.scraper()

  0%|          | 0/29 [00:00<?, ?it/s]


HTTPError: HTTP Error 403: Forbidden

In [13]:
!pip install investpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 52.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/anaconda/envs/azureml_py38/lib/python3.10/site-packages/numpy-1.23.5.dist-info/METADATA'



In [2]:
!pip install yfinance

  Using cached yfinance-0.2.55-py2.py3-none-any.whl.metadata (5.8 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
  Using cached frozendict-2.4.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (23 kB)
  Using cached peewee-3.17.9.tar.gz (3.0 MB)
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/anaconda/envs/azureml_py38/lib/python3.10/site-packages/numpy-1.23.5.dist-info/METADATA'



In [14]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

def get_forex_news_five_years(currency_pair):

    headers = {"User-Agent": "Mozilla/5.0"}
    articles = []
    
    page_count = 0
    stop_flag = False
    article_titles = []
    article_dates = []
    while not stop_flag and page_count <= 10:
        if page_count == 0:
            url = 'https://www.investing.com/currencies/'+currency_pair+'-news'
        else:
            url = 'https://www.investing.com/currencies/'+currency_pair+'-news/'+str(page_count)
        response = requests.get(url, headers=headers)
        print('url:',url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            news_list = soup.find('ul', attrs={'data-test': 'news-list'})

            # Extract all <article> elements inside it
            articles = news_list.find_all('article')
            for article in articles:
                div_tag = article.find('div', class_='block w-full sm:flex-1')
                if div_tag:
                    # Extract the <a> tag inside the <div>
                    a_tag = div_tag.find('a', href=True)
                else:
                    print("coudln't find div tag of class block w-full sm:flex-1")
                    assert(False)
                if a_tag:
                    article_title = a_tag.get_text(strip=True)
                    
                else:
                    print("coudln't find a tag with href")
                    assert(False)

                time_tag = div_tag.find('time', datetime=True)

                if time_tag:
                    # Get the datetime attribute from the <time> tag
                    datetime_value = time_tag['datetime'].split()[0]
                    dt = datetime.strptime(datetime_value, '%Y-%m-%d').date()
                    current_date = datetime.now().date()
                    diff_in_years = current_date.year - dt.year
                    if diff_in_years == 5:
                        stop_flag = True
                    else:
                        article_titles.append(article_title)
                        article_dates.append(datetime_value)


                else:
                    print("No <time> tag with datetime attribute found.")
                    assert(False)

        else:
            print("couldn't get page info:",url)
            assert(False)

        page_count += 1

    df = pd.DataFrame({
    'Title': article_titles,
    'Date': article_dates
    })
    c1 = currency_pair.split('-')[0]
    c2 = currency_pair.split('-')[1]
    str_ = c1+'_'+c2
    df.to_csv('currency_pair_info'+str_+'.csv', index=False)
    print("saved csv")
    print("done")

currency_pair = 'gbp-usd'
get_forex_news_five_years(currency_pair)


ModuleNotFoundError: No module named 'investpy'

In [14]:
import datetime
a = datetime.date(2011,11,24)
b = datetime.date(2011,11,17)
(a-b).days
start_date = '2021-01-04'
day = 'mon'
date_format = "%Y-%m-%d"
today = pd.to_datetime(start_date, format=date_format).date()
today

datetime.date(2021, 1, 4)

In [68]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt

def build_dataset(price_csv, titles_csv, pred_horizon, dest_file):

    df_price = pd.read_csv(price_csv)
    df_titles = pd.read_csv(titles_csv)

    df_titles['Date'] = pd.to_datetime(df_titles['Date']).dt.date  # Converts to datetime.date
    df_price['Date'] = pd.to_datetime(df_price['Date']).dt.date

    date_format = "%Y-%m-%d"
    dates = df_price['Date'].tolist()
    titles_col = []
    label_col = []
    for datestr in dates:
        today = pd.to_datetime(datestr, format=date_format).date()
        next_day_dt = today + datetime.timedelta(days=1)
        next_day = next_day_dt.strftime(date_format)
        #pd.to_datetime(df_titles['Date'], format=date_format).date()
        #assert(False)
        df1 = df_titles[(today - df_titles['Date'] >= datetime.timedelta(days=0)) & (today - df_titles['Date'] < datetime.timedelta(days=pred_horizon))]
        relevant_titles = df_titles[(today - df_titles['Date'] >= datetime.timedelta(days=0)) & (today - df_titles['Date'] < datetime.timedelta(days=pred_horizon))]['Title'].tolist()
        today_price = df_price[df_price['Date'] == today]['Close'].tolist()[0]
        print('today price:',today_price)
        try:
            tomorrow_price = df_price[df_price['Date'] == next_day]['Close'].tolist()[0]
        except:
            # case when next day is a Saturday
            next_day_dt = today + datetime.timedelta(days=3)
            next_day = next_day_dt.strftime(date_format)
            tomorrow_price = df_price[df_price['Date'] == next_day]['Close'].tolist()[0]

        print('tomorrow price:',tomorrow_price)
        label = int(tomorrow_price > today_price)

        titles = "\n".join(relevant_titles)

        titles_col.append(titles)
        label_col.append(label)

    
    df = pd.DataFrame({
    'Titles': titles_col,
    'Label': label_col
    })

    df.to_csv(dest_file, index=False)

price_csv = 'gbp_usd.csv'
titles_csv = 'currency_pair_infogbp_usd.csv' 
pred_horizon = 5
dest_file = "gbp_usd_dataset.csv"

build_dataset(price_csv, titles_csv, pred_horizon, dest_file)

today price: 1.3684196472167969


AssertionError: 

In [56]:
import matplotlib.pyplot as plt
df_dataset = pd.read_csv('gbp_usd_dataset.csv')
L = df_dataset['Label'].tolist()
print(L.count(1))
print(L.count(0))

177
931
